In [1]:
import os
import numpy as np
import pandas as pd
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

2023-08-24 18:46:50.805402: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-24 18:46:55.444727: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-08-24 18:46:55.446799: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [2]:
# Load metadata CSV file
metadata = pd.read_csv('HAM10000_metadata.csv')

In [3]:
# Load images and labels
image_folder = 'HAM10000_Images'  # Replace with the actual image folder path
# image_size = (224, 224)
# image_size = (32, 32)
image_size = (96,96)

In [4]:
images = []
labels = []

for index, row in metadata.iterrows():
    image_path = os.path.join(image_folder, row['image_id'] + '.jpg')
    image = imread(image_path)
    resized_image = resize(image, image_size)
    images.append(resized_image)
    labels.append(row['dx'])


In [ ]:
images = np.array(images)
labels = np.array(labels)

In [6]:
# Convert labels to categorical format
label_mapping = {label: idx for idx, label in enumerate(np.unique(labels))}
labels_encoded = np.array([label_mapping[label] for label in labels])
labels_categorical = to_categorical(labels_encoded, num_classes=len(label_mapping))

In [7]:

# Split the dataset into train, validation, and test sets
train_images, test_images, train_labels, test_labels = train_test_split(
    images, labels_categorical, test_size=0.2, random_state=42
)
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.2, random_state=42
)


In [8]:

# Print the shape of each set
print("Train images shape:", train_images.shape)
print("Train labels shape:", train_labels.shape)
print("Validation images shape:", val_images.shape)
print("Validation labels shape:", val_labels.shape)
print("Test images shape:", test_images.shape)
print("Test labels shape:", test_labels.shape)


Train images shape: (6409, 96, 96, 3)
Train labels shape: (6409, 7)
Validation images shape: (1603, 96, 96, 3)
Validation labels shape: (1603, 7)
Test images shape: (2003, 96, 96, 3)
Test labels shape: (2003, 7)


In [9]:
# Create an ImageDataGenerator instance for data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,  # Random rotation up to 20 degrees
    width_shift_range=0.1,  # Random horizontal shift
    height_shift_range=0.1,  # Random vertical shift
    shear_range=0.2,  # Shear transformations
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Random horizontal flips
    vertical_flip=True,  # Random vertical flips
    fill_mode='nearest'  # Fill points outside the input with the nearest pixel
)

In [10]:
# Apply data augmentation only to the training set
train_datagen = datagen.flow(train_images, train_labels, batch_size=32, shuffle=True)

In [11]:
# Load the pre-trained VGG16 model (excluding top layers and with ImageNet weights)
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(96, 96, 3))

2023-08-19 21:05:02.383518: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-08-19 21:05:02.383575: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-08-19 21:05:02.383617: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (instance-20230816-171017): /proc/driver/nvidia/version does not exist
2023-08-19 21:05:02.385145: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow 

In [12]:

# Freeze the layers of the base model to retain pre-trained weights
for layer in base_model.layers:
    layer.trainable = False

In [13]:
# Add custom classification layers on top of the base model
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(7, activation='softmax')(x)

In [14]:
# Create the final model
model = Model(inputs=base_model.input, outputs=output)

In [15]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [16]:
from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping criteria
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor the validation loss
    patience=10,          # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore the best model weights based on validation loss
)


In [17]:
# Train the model using the fit() method
epochs = 40  # Number of training epochs
batch_size = 32  # Number of samples in each batch

history = model.fit(
    train_datagen,  # If using data augmentation
    # x=train_images, y=train_labels,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(val_images, val_labels),
    callbacks=[early_stopping]
)


Epoch 1/40
201/201 [==============================] - 394s 2s/step - loss: 1.3092 - accuracy: 0.6166 - val_loss: 1.0872 - val_accuracy: 0.6606
Epoch 2/40
201/201 [==============================] - 389s 2s/step - loss: 1.0483 - accuracy: 0.6725 - val_loss: 0.9925 - val_accuracy: 0.6656
Epoch 3/40
201/201 [==============================] - 389s 2s/step - loss: 0.9999 - accuracy: 0.6744 - val_loss: 0.9504 - val_accuracy: 0.6750
Epoch 4/40
201/201 [==============================] - 386s 2s/step - loss: 0.9739 - accuracy: 0.6789 - val_loss: 0.9274 - val_accuracy: 0.6831
Epoch 5/40
201/201 [==============================] - 390s 2s/step - loss: 0.9499 - accuracy: 0.6829 - val_loss: 0.9105 - val_accuracy: 0.6887
Epoch 6/40
201/201 [==============================] - 388s 2s/step - loss: 0.9314 - accuracy: 0.6897 - val_loss: 0.9025 - val_accuracy: 0.7006
Epoch 7/40
201/201 [==============================] - 388s 2s/step - loss: 0.9188 - accuracy: 0.6889 - val_loss: 0.8848 - val_accuracy: 0.7031

In [18]:
# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(val_images, val_labels, batch_size=batch_size)

print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)


51/51 [==============================] - 76s 1s/step - loss: 0.7903 - accuracy: 0.7274
Validation Loss: 0.7902973890304565
Validation Accuracy: 0.7273861765861511


In [19]:
# Evaluate the model on the testing set
test_loss, test_accuracy = model.evaluate(test_images, test_labels, batch_size=batch_size)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


63/63 [==============================] - 95s 2s/step - loss: 0.8035 - accuracy: 0.7099
Test Loss: 0.803539514541626
Test Accuracy: 0.7099350690841675


In [20]:
model.save('model_96x96')

INFO:tensorflow:Assets written to: model_96x96/assets


INFO:tensorflow:Assets written to: model_96x96/assets


In [54]:
print(label_mapping)

{'akiec': 0, 'bcc': 1, 'bkl': 2, 'df': 3, 'mel': 4, 'nv': 5, 'vasc': 6}
